In [1]:
import math
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import time
import datetime
import random
random.seed(1234)

from scipy import interp
import warnings
warnings.filterwarnings("ignore")

from collections import Counter
from functools import reduce
from tqdm import tqdm, trange
from copy import deepcopy

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import classification_report
from sklearn.utils import class_weight

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
#设置value的显示长度为200，默认为50
# pd.set_option('max_colwidth',200)
# #显示所有列，把行显示设置成最大
# pd.set_option('display.max_columns', None)
#显示所有行，把列显示设置成最大
pd.set_option('display.max_rows', 500)

In [3]:
seed = 19961231
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [4]:
hla_sequence = pd.read_csv('/home/chujunyi/5_ZY_MHC/data/common_hla_sequence.csv')

In [5]:
def make_data(data):
    pep_inputs, hla_inputs, labels = [], [], []
    for pep, hla, label in zip(data.peptide, data.HLA_sequence, data.label):
        pep, hla = pep.ljust(pep_max_len, '-'), hla.ljust(hla_max_len, '-')
        pep_input = [[vocab[n] for n in pep]] # [[1, 2, 3, 4, 0], [1, 2, 3, 5, 0]]
        hla_input = [[vocab[n] for n in hla]]
        pep_inputs.extend(pep_input)
        hla_inputs.extend(hla_input)
        labels.append(label)
    return torch.LongTensor(pep_inputs), torch.LongTensor(hla_inputs), torch.LongTensor(labels)

class MyDataSet(Data.Dataset):
    def __init__(self, pep_inputs, hla_inputs, labels):
        super(MyDataSet, self).__init__()
        self.pep_inputs = pep_inputs
        self.hla_inputs = hla_inputs
        self.labels = labels

    def __len__(self): # 样本数
        return self.pep_inputs.shape[0] # 改成hla_inputs也可以哦！

    def __getitem__(self, idx):
        return self.pep_inputs[idx], self.hla_inputs[idx], self.labels[idx]

## 模型

In [6]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        '''
        x: [seq_len, batch_size, d_model]
        '''
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [7]:
def get_attn_pad_mask(seq_q, seq_k):
    '''
    seq_q: [batch_size, seq_len]
    seq_k: [batch_size, seq_len]
    seq_len could be src_len or it could be tgt_len
    seq_len in seq_q and seq_len in seq_k maybe not equal
    '''
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # [batch_size, 1, len_k], False is masked
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # [batch_size, len_q, len_k]

In [8]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        '''
        Q: [batch_size, n_heads, len_q, d_k]
        K: [batch_size, n_heads, len_k, d_k]
        V: [batch_size, n_heads, len_v(=len_k), d_v]
        attn_mask: [batch_size, n_heads, seq_len, seq_len]
        '''
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size, n_heads, len_q, len_k]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is True.
        
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V) # [batch_size, n_heads, len_q, d_v]
        return context, attn

In [9]:
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device("cuda" if self.use_cuda else "cpu")
        self.W_Q = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_K = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_V = nn.Linear(d_model, d_v * n_heads, bias=False)
        self.fc = nn.Linear(n_heads * d_v, d_model, bias=False)
    def forward(self, input_Q, input_K, input_V, attn_mask):
        '''
        input_Q: [batch_size, len_q, d_model]
        input_K: [batch_size, len_k, d_model]
        input_V: [batch_size, len_v(=len_k), d_model]
        attn_mask: [batch_size, seq_len, seq_len]
        '''
        residual, batch_size = input_Q, input_Q.size(0)
        # (B, S, D) -proj-> (B, S, D_new) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        Q = self.W_Q(input_Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # Q: [batch_size, n_heads, len_q, d_k]
        K = self.W_K(input_K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # K: [batch_size, n_heads, len_k, d_k]
        V = self.W_V(input_V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # V: [batch_size, n_heads, len_v(=len_k), d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size, n_heads, seq_len, seq_len]

        # context: [batch_size, n_heads, len_q, d_v], attn: [batch_size, n_heads, len_q, len_k]
        context, attn = ScaledDotProductAttention()(Q, K, V, attn_mask)
        context = context.transpose(1, 2).reshape(batch_size, -1, n_heads * d_v) # context: [batch_size, len_q, n_heads * d_v]
        output = self.fc(context) # [batch_size, len_q, d_model]
        return nn.LayerNorm(d_model).to(device)(output + residual), attn

In [10]:
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device("cuda" if self.use_cuda else "cpu")
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=False),
            nn.ReLU(),
            nn.Linear(d_ff, d_model, bias=False)
        )
    def forward(self, inputs):
        '''
        inputs: [batch_size, seq_len, d_model]
        '''
        residual = inputs
        output = self.fc(inputs)
        return nn.LayerNorm(d_model).to(device)(output + residual) # [batch_size, seq_len, d_model]

In [11]:
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        '''
        enc_inputs: [batch_size, src_len, d_model]
        enc_self_attn_mask: [batch_size, src_len, src_len]
        '''
        # enc_outputs: [batch_size, src_len, d_model], attn: [batch_size, n_heads, src_len, src_len]
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size, src_len, d_model]
        return enc_outputs, attn

In [12]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.src_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

    def forward(self, enc_inputs):
        '''
        enc_inputs: [batch_size, src_len]
        '''
        enc_outputs = self.src_emb(enc_inputs) # [batch_size, src_len, d_model]
        enc_outputs = self.pos_emb(enc_outputs.transpose(0, 1)).transpose(0, 1) # [batch_size, src_len, d_model]
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs) # [batch_size, src_len, src_len]
        enc_self_attns = []
        for layer in self.layers:
            # enc_outputs: [batch_size, src_len, d_model], enc_self_attn: [batch_size, n_heads, src_len, src_len]
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)
            enc_self_attns.append(enc_self_attn)
        return enc_outputs, enc_self_attns

### Decoder

In [13]:
class DecoderLayer(nn.Module):
    def __init__(self):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, dec_inputs, dec_self_attn_mask): # dec_inputs = enc_outputs
        '''
        dec_inputs: [batch_size, tgt_len, d_model]
        enc_outputs: [batch_size, src_len, d_model]
        dec_self_attn_mask: [batch_size, tgt_len, tgt_len]
        '''
        # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len]
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)
        dec_outputs = self.pos_ffn(dec_outputs) # [batch_size, tgt_len, d_model]
        return dec_outputs, dec_self_attn

In [14]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
#         self.tgt_emb = nn.Embedding(d_model * 2, d_model)
        self.use_cuda = use_cuda
        device = torch.device("cuda" if self.use_cuda else "cpu")
        self.pos_emb = PositionalEncoding(d_model)
        self.layers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)])
        self.tgt_len = tgt_len
        
    def forward(self, dec_inputs): # dec_inputs = enc_outputs (batch_size, peptide_hla_maxlen_sum, d_model)
        '''
        dec_inputs: [batch_size, tgt_len]
        enc_intpus: [batch_size, src_len]
        enc_outputs: [batsh_size, src_len, d_model]
        '''
#         dec_outputs = self.tgt_emb(dec_inputs) # [batch_size, tgt_len, d_model]
        dec_outputs = self.pos_emb(dec_inputs.transpose(0, 1)).transpose(0, 1).to(device) # [batch_size, tgt_len, d_model]
#         dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs).cuda() # [batch_size, tgt_len, tgt_len]
        dec_self_attn_pad_mask = torch.LongTensor(np.zeros((dec_inputs.shape[0], tgt_len, tgt_len))).bool().to(device)

        dec_self_attns = []
        for layer in self.layers:
            # dec_outputs: [batch_size, tgt_len, d_model], dec_self_attn: [batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [batch_size, h_heads, tgt_len, src_len]
            dec_outputs, dec_self_attn = layer(dec_outputs, dec_self_attn_pad_mask)
            dec_self_attns.append(dec_self_attn)
            
        return dec_outputs, dec_self_attns

### Transformer

In [15]:
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device("cuda" if use_cuda else "cpu")
        self.pep_encoder = Encoder().to(device)
        self.hla_encoder = Encoder().to(device)
        self.decoder = Decoder().to(device)
        self.tgt_len = tgt_len
        self.projection = nn.Sequential(
                                        nn.Linear(tgt_len * d_model, 256),
                                        nn.ReLU(True),

                                        nn.BatchNorm1d(256),
                                        nn.Linear(256, 64),
                                        nn.ReLU(True),

                                        #output layer
                                        nn.Linear(64, 2)
                                        ).to(device)
        
    def forward(self, pep_inputs, hla_inputs):
        '''
        pep_inputs: [batch_size, pep_len]
        hla_inputs: [batch_size, hla_len]
        '''
        # tensor to store decoder outputs
        # outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)
        
        # enc_outputs: [batch_size, src_len, d_model], enc_self_attns: [n_layers, batch_size, n_heads, src_len, src_len]
        pep_enc_outputs, pep_enc_self_attns = self.pep_encoder(pep_inputs)
        hla_enc_outputs, hla_enc_self_attns = self.hla_encoder(hla_inputs)
        enc_outputs = torch.cat((pep_enc_outputs, hla_enc_outputs), 1) # concat pep & hla embedding
        
        # dec_outpus: [batch_size, tgt_len, d_model], dec_self_attns: [n_layers, batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [n_layers, batch_size, tgt_len, src_len]
        dec_outputs, dec_self_attns = self.decoder(enc_outputs)
        dec_outputs = dec_outputs.view(dec_outputs.shape[0], -1) # Flatten [batch_size, tgt_len * d_model]
        dec_logits = self.projection(dec_outputs) # dec_logits: [batch_size, tgt_len, tgt_vocab_size]

        return dec_logits.view(-1, dec_logits.size(-1)), pep_enc_self_attns, hla_enc_self_attns, dec_self_attns

# 性能评估

In [16]:
def performances(y_true, y_pred, y_prob, print_ = True):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    try:
        mcc = ((tp*tn) - (fn*fp)) / np.sqrt(np.float((tp+fn)*(tn+fp)*(tp+fp)*(tn+fn)))
    except:
        print('MCC Error: ', (tp+fn)*(tn+fp)*(tp+fp)*(tn+fn))
        mcc = np.nan
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    try:
        recall = tp / (tp+fn)
    except:
        recall = np.nan
        
    try:
        precision = tp / (tp+fp)
    except:
        precision = np.nan
        
    try: 
        f1 = 2*precision*recall / (precision+recall)
    except:
        f1 = np.nan
        
    roc_auc = roc_auc_score(y_true, y_prob)
    prec, reca, _ = precision_recall_curve(y_true, y_prob)
    aupr = auc(reca, prec)
    
    if print_:
        print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
        print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
        print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
        print('auc={:.4f}|sensitivity={:.4f}|specificity={:.4f}|acc={:.4f}|mcc={:.4f}'.format(roc_auc, sensitivity, specificity, accuracy, mcc))
        print('precision={:.4f}|recall={:.4f}|f1={:.4f}|aupr={:.4f}'.format(precision, recall, f1, aupr))
    
    return (roc_auc, accuracy, mcc, f1, sensitivity, specificity, precision, recall, aupr)

In [17]:
def transfer(y_prob, threshold = 0.5):
    return np.array([[0, 1][x > threshold] for x in y_prob])

In [18]:
def performances_to_pd(performances_dict):
    metrics_name = ['roc_auc', 'accuracy', 'mcc', 'f1', 'sensitivity', 'specificity', 'precision', 'recall', 'aupr']
    performances_pd = pd.DataFrame(performances_dict, index = metrics_name)
    return performances_pd.T

### 模型 & 损失函数 & 优化器

In [19]:
def eval_step(model, val_loader, labels, fold, use_cuda = False, save_ = False):
    device = torch.device("cuda" if use_cuda else "cpu")
    
    model.eval()
    torch.manual_seed(19961231)
    torch.cuda.manual_seed(19961231)
    with torch.no_grad():
        y_true_val_list, y_prob_val_list, dec_attns_val_list = [], [], []
        for val_pep_inputs, val_hla_inputs, val_labels in tqdm(val_loader):
            val_pep_inputs, val_hla_inputs, val_labels = val_pep_inputs.to(device), val_hla_inputs.to(device), val_labels.to(device)
            val_outputs, _, _, val_dec_self_attns = model(val_pep_inputs, val_hla_inputs)
            val_loss = criterion(val_outputs, val_labels)

            y_true_val = val_labels.cpu().numpy()
            y_prob_val = nn.Softmax(dim = 1)(val_outputs)[:, 1].cpu().detach().numpy()

            y_true_val_list.extend(y_true_val)
            y_prob_val_list.extend(y_prob_val)
            
            if save_:
                dec_attns_val_list.extend(val_dec_self_attns[0][:, :, 15:, :15]) # 只要（34,15）行HLA，列peptide
                
        assert (labels.numpy() == y_true_val_list).all()    
        y_pred_val_list = transfer(y_prob_val_list, threshold)
        metrics_val = performances(y_true_val_list, y_pred_val_list, y_prob_val_list, print_ = True)
        
        if save_: 
            ys_val = (y_true_val_list, y_pred_val_list, y_prob_val_list)
            return metrics_val, ys_val, dec_attns_val_list
        else:
            return metrics_val, None, None

In [20]:
def data_with_loader(type_ = 'train',fold = None,  batch_size = 1024):
    if type_ != 'train' and type_ != 'val':
        data = pd.read_csv('/home/chujunyi/5_ZY_MHC/Anthem/Dataset/{}_set.csv'.format(type_), index_col = 0)
    elif type_ == 'train':
        data = pd.read_csv('/home/chujunyi/5_ZY_MHC/Anthem/Dataset/train_data_fold{}.csv'.format(fold), index_col = 0)
    elif type_ == 'val':
        data = pd.read_csv('/home/chujunyi/5_ZY_MHC/Anthem/Dataset/val_data_fold{}.csv'.format(fold), index_col = 0)
        
    pep_inputs, hla_inputs, labels = make_data(data)
    if batch_size != None:
        loader = Data.DataLoader(MyDataSet(pep_inputs, hla_inputs, labels), batch_size, shuffle = False, num_workers = 0)
    else:
        loader = Data.DataLoader(MyDataSet(pep_inputs, hla_inputs, labels), batch_size = len(data), shuffle = False, num_workers = 0)
    return data, pep_inputs, hla_inputs, labels, loader

# 运行

In [21]:
pep_max_len = 15 # peptide; enc_input max sequence length
hla_max_len = 34 # hla; dec_input(=dec_output) max sequence length
tgt_len = pep_max_len + hla_max_len
pep_max_len, hla_max_len

vocab = np.load('Transformer_vocab_dict.npy', allow_pickle = True).item()
vocab_size = len(vocab)


# In[36]:


# Transformer Parameters
d_model = 64  # Embedding Size
d_ff = 512 # FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
# n_layers = 1  # number of Encoder of Decoder Layer

batch_size = 1024
epochs = 50
threshold = 0.5

In [22]:
model_files = sorted(os.listdir('/home/chujunyi/5_ZY_MHC/model/pHLAIformer'), key = lambda x : (x.split('_')[1][-1], eval(x.split('_')[2][9:]), eval(x.split('_')[3][4])))
# model_files = []
# for f in files:
#     if f.split('_')[3][4] == '0': model_files.append(f)
model_layers_heads = [[item.split('_')[1][5:], item.split('_')[2][9:], item.split('_')[-1][4:-4]] for item in model_files]
model_layers_heads

[['1', '1', '0'],
 ['1', '1', '1'],
 ['1', '1', '2'],
 ['1', '1', '3'],
 ['1', '1', '4'],
 ['1', '2', '0'],
 ['1', '2', '1'],
 ['1', '2', '2'],
 ['1', '2', '3'],
 ['1', '2', '4'],
 ['1', '3', '0'],
 ['1', '3', '1'],
 ['1', '3', '2'],
 ['1', '3', '3'],
 ['1', '3', '4'],
 ['1', '4', '0'],
 ['1', '4', '1'],
 ['1', '4', '2'],
 ['1', '4', '3'],
 ['1', '4', '4'],
 ['1', '5', '0'],
 ['1', '5', '1'],
 ['1', '5', '2'],
 ['1', '5', '3'],
 ['1', '5', '4'],
 ['1', '6', '0'],
 ['1', '6', '1'],
 ['1', '6', '2'],
 ['1', '6', '3'],
 ['1', '6', '4'],
 ['1', '7', '0'],
 ['1', '7', '1'],
 ['1', '7', '2'],
 ['1', '7', '3'],
 ['1', '7', '4'],
 ['1', '8', '0'],
 ['1', '8', '1'],
 ['1', '8', '2'],
 ['1', '8', '3'],
 ['1', '8', '4'],
 ['1', '9', '0'],
 ['1', '9', '1'],
 ['1', '9', '2'],
 ['1', '9', '3'],
 ['1', '9', '4'],
 ['1', '10', '0'],
 ['1', '10', '1'],
 ['1', '10', '2'],
 ['1', '10', '3'],
 ['1', '10', '4'],
 ['2', '1', '0'],
 ['2', '1', '1'],
 ['2', '1', '2'],
 ['2', '1', '3'],
 ['2', '1', '4'],
 ['2'

In [22]:
type_ = 'independent'
save_ = False
use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

independent_metrics_dict, independent_ys_dict, independent_attns_dict = {}, {}, {}
for idx, filename in enumerate(model_files):
    n_layers, n_heads, fold = model_layers_heads[idx]
    n_layers, n_heads, fold = int(n_layers), int(n_heads), int(fold)
    temp =  str(n_layers) + '-' + str(n_heads) + '-' + str(fold)
    print('Idx{} - {}: # layer = {} | # heads = {} | Fold = {}'.format(idx, filename, n_layers, n_heads, fold))
    
    model_eval = Transformer().to(device)
    criterion = nn.CrossEntropyLoss()
    model_pwd = '/home/chujunyi/5_ZY_MHC/model/pHLAIformer/'
    model_eval.load_state_dict(torch.load(model_pwd + filename), strict = True)

    if type_ == 'train' or type_ == 'val':
        data, pep_inputs, hla_inputs, labels, loader = data_with_loader(type_, fold,  batch_size)
    else:
        data, pep_inputs, hla_inputs, labels, loader = data_with_loader(type_, None, batch_size)
    print('{} Label info: {}'.format(str.upper(type_), Counter(data.label)))

    model_eval.eval()
    if save_:
        independent_metrics_res, independent_ys_res, independent_attn_res = eval_step(model_eval, loader, labels, fold, type_, use_cuda, save_)
        independent_metrics_dict[temp], independent_ys_dict[temp], independent_attns_dict[temp] = independent_metrics_res, independent_ys_res, independent_attn_res
    else:
        independent_metrics_res, _, _ = eval_step(model_eval, loader, labels, fold, type_, use_cuda, save_)
        independent_metrics_dict[temp] = independent_metrics_res
independent_metrics_pd = performances_to_pd(independent_metrics_dict)

Idx0 - model_layer1_multihead1_fold0.pkl: # layer = 1 | # heads = 1 | Fold = 0


  1%|          | 2/168 [00:00<00:12, 13.65it/s]

INDEPENDENT Label info: Counter({1: 85876, 0: 85562})


100%|██████████| 168/168 [00:09<00:00, 16.96it/s]


tn = 79342, fp = 6220, fn = 5610, tp = 80266
y_pred: 0 = 84952 | 1 = 86486
y_true: 0 = 85562 | 1 = 85876
auc=0.9776|sensitivity=0.9347|specificity=0.9273|acc=0.9310|mcc=0.8620
precision=0.9281|recall=0.9347|f1=0.9314|aupr=0.9775
Idx1 - model_layer1_multihead1_fold1.pkl: # layer = 1 | # heads = 1 | Fold = 1


...

tn = 79408, fp = 6154, fn = 5985, tp = 79891
y_pred: 0 = 85393 | 1 = 86045
y_true: 0 = 85562 | 1 = 85876
auc=0.9784|sensitivity=0.9303|specificity=0.9281|acc=0.9292|mcc=0.8584
precision=0.9285|recall=0.9303|f1=0.9294|aupr=0.9778
Idx99 - model_layer2_multihead10_fold4.pkl: # layer = 2 | # heads = 10 | Fold = 4


  0%|          | 0/168 [00:00<?, ?it/s]

INDEPENDENT Label info: Counter({1: 85876, 0: 85562})


100%|██████████| 168/168 [00:43<00:00,  3.84it/s]


tn = 79235, fp = 6327, fn = 5536, tp = 80340
y_pred: 0 = 84771 | 1 = 86667
y_true: 0 = 85562 | 1 = 85876
auc=0.9789|sensitivity=0.9355|specificity=0.9261|acc=0.9308|mcc=0.8616
precision=0.9270|recall=0.9355|f1=0.9312|aupr=0.9781


In [31]:
independent_metrics_pd

,roc_auc,accuracy,mcc,f1,sensitivity,specificity,precision,recall,aupr
1-1-0,0.977645,0.930995,0.862010,0.931365,0.934673,0.927304,0.928081,0.934673,0.977522
1-1-1,0.977057,0.928948,0.857945,0.929468,0.934603,0.923272,0.924388,0.934603,0.976965
1-1-2,0.978314,0.929625,0.859332,0.930247,0.936828,0.922395,0.923758,0.936828,0.978033
1-1-3,0.977433,0.928802,0.857605,0.929008,0.930004,0.927596,0.928015,0.930004,0.977332
1-1-4,0.977227,0.929403,0.858809,0.929641,0.931075,0.927725,0.928211,0.931075,0.977057
1-2-0,0.977629,0.928866,0.857915,0.929732,0.939459,0.918235,0.920203,0.939459,0.977445
1-2-1,0.976449,0.926708,0.853453,0.927194,0.931681,0.921718,0.922751,0.931681,0.976295
1-2-2,0.977907,0.928750,0.857568,0.929338,0.935360,0.922115,0.923393,0.935360,0.977481
1-2-3,0.977490,0.928511,0.857065,0.929014,0.933893,0.923108,0.924186,0.933893,0.976986
1-2-4,0.977822,0.930535,0.861076,0.930811,0.932822,0.928239,0.928809,0.932822,0.977289


In [24]:
type_ = 'external'
save_ = False
use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

external_metrics_dict, external_ys_dict, external_attns_dict = {}, {}, {}
for idx, filename in enumerate(model_files):
    n_layers, n_heads, fold = model_layers_heads[idx]
    n_layers, n_heads, fold = int(n_layers), int(n_heads), int(fold)
    temp =  str(n_layers) + '-' + str(n_heads) + '-' + str(fold)
    print('Idx{} - {}: # layer = {} | # heads = {} | Fold = {}'.format(idx, filename, n_layers, n_heads, fold))
    
    model_eval = Transformer().to(device)
    criterion = nn.CrossEntropyLoss()
    model_pwd = '/home/chujunyi/5_ZY_MHC/model/pHLAIformer/'
    model_eval.load_state_dict(torch.load(model_pwd + filename), strict = True)

    if type_ == 'train' or type_ == 'val':
        data, pep_inputs, hla_inputs, labels, loader = data_with_loader(type_, fold,  batch_size)
    else:
        data, pep_inputs, hla_inputs, labels, loader = data_with_loader(type_, None, batch_size)
    print('{} Label info: {}'.format(str.upper(type_), Counter(data.label)))

    model_eval.eval()
    if save_:
        external_metrics_res, external_ys_res, external_attn_res = eval_step(model_eval, loader, labels, fold, type_, use_cuda, save_)
        external_metrics_dict[temp], external_ys_dict[temp], external_attns_dict[temp] = external_metrics_res, external_ys_res, external_attn_res
    else:
        external_metrics_res, _, _ = eval_step(model_eval, loader, labels, fold, type_, use_cuda, save_)
        external_metrics_dict[temp] = external_metrics_res
external_metrics_pd = performances_to_pd(external_metrics_dict)

Idx0 - model_layer1_multihead1_fold0.pkl: # layer = 1 | # heads = 1 | Fold = 0


  2%|▏         | 2/102 [00:00<00:06, 16.34it/s]

EXTERNAL Label info: Counter({1: 51984, 0: 51881})


100%|██████████| 102/102 [00:05<00:00, 18.60it/s]


tn = 48607, fp = 3274, fn = 8705, tp = 43279
y_pred: 0 = 57312 | 1 = 46553
y_true: 0 = 51881 | 1 = 51984
auc=0.9543|sensitivity=0.8325|specificity=0.9369|acc=0.8847|mcc=0.7736
precision=0.9297|recall=0.8325|f1=0.8784|aupr=0.9571
Idx1 - model_layer1_multihead1_fold1.pkl: # layer = 1 | # heads = 1 | Fold = 1


...

tn = 49346, fp = 2535, fn = 10794, tp = 41190
y_pred: 0 = 60140 | 1 = 43725
y_true: 0 = 51881 | 1 = 51984
auc=0.9515|sensitivity=0.7924|specificity=0.9511|acc=0.8717|mcc=0.7530
precision=0.9420|recall=0.7924|f1=0.8607|aupr=0.9545
Idx99 - model_layer2_multihead10_fold4.pkl: # layer = 2 | # heads = 10 | Fold = 4


  0%|          | 0/102 [00:00<?, ?it/s]

EXTERNAL Label info: Counter({1: 51984, 0: 51881})


100%|██████████| 102/102 [00:26<00:00,  3.84it/s]


tn = 48516, fp = 3365, fn = 8878, tp = 43106
y_pred: 0 = 57394 | 1 = 46471
y_true: 0 = 51881 | 1 = 51984
auc=0.9500|sensitivity=0.8292|specificity=0.9351|acc=0.8821|mcc=0.7686
precision=0.9276|recall=0.8292|f1=0.8756|aupr=0.9544


In [32]:
external_metrics_pd

,roc_auc,accuracy,mcc,f1,sensitivity,specificity,precision,recall,aupr
1-1-0,0.954303,0.884668,0.773600,0.878431,0.832545,0.936894,0.929672,0.832545,0.957119
1-1-1,0.954155,0.886275,0.775820,0.880951,0.840720,0.931921,0.925227,0.840720,0.957109
1-1-2,0.954739,0.890213,0.782496,0.886206,0.854147,0.926351,0.920764,0.854147,0.957623
1-1-3,0.954440,0.885736,0.776273,0.879164,0.830525,0.941057,0.933855,0.830525,0.957218
1-1-4,0.950397,0.876850,0.759716,0.868758,0.814385,0.939438,0.930910,0.814385,0.953704
1-2-0,0.950416,0.884225,0.771450,0.879033,0.840470,0.928066,0.921304,0.840470,0.955023
1-2-1,0.950564,0.871420,0.751490,0.861069,0.796130,0.946859,0.937544,0.796130,0.952601
1-2-2,0.949817,0.873865,0.754401,0.865068,0.807864,0.939997,0.930989,0.807864,0.952447
1-2-3,0.952646,0.875502,0.758832,0.866060,0.804209,0.946936,0.938217,0.804209,0.955500
1-2-4,0.952546,0.883791,0.772386,0.877088,0.828428,0.939265,0.931820,0.828428,0.955263


In [26]:
type_ = 'val'
save_ = False
use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

val_metrics_dict, val_ys_dict, val_attns_dict = {}, {}, {}
for idx, filename in enumerate(model_files):
    n_layers, n_heads, fold = model_layers_heads[idx]
    n_layers, n_heads, fold = int(n_layers), int(n_heads), int(fold)
    temp =  str(n_layers) + '-' + str(n_heads) + '-' + str(fold)
    print('Idx{} - {}: # layer = {} | # heads = {} | Fold = {}'.format(idx, filename, n_layers, n_heads, fold))
    
    model_eval = Transformer().to(device)
    criterion = nn.CrossEntropyLoss()
    model_pwd = '/home/chujunyi/5_ZY_MHC/model/pHLAIformer/'
    model_eval.load_state_dict(torch.load(model_pwd + filename), strict = True)

    if type_ == 'train' or type_ == 'val':
        data, pep_inputs, hla_inputs, labels, loader = data_with_loader(type_, fold,  batch_size)
    else:
        data, pep_inputs, hla_inputs, labels, loader = data_with_loader(type_, None, batch_size)
    print('{} Label info: {}'.format(str.upper(type_), Counter(data.label)))

    model_eval.eval()
    if save_:
        val_metrics_res, val_ys_res, val_attn_res = eval_step(model_eval, loader, labels, fold, type_, use_cuda, save_)
        val_metrics_dict[temp], val_ys_dict[temp], val_attns_dict[temp] = val_metrics_res, val_ys_res, val_attn_res
    else:
        val_metrics_res, _, _ = eval_step(model_eval, loader, labels, fold, type_, use_cuda, save_)
        val_metrics_dict[temp] = val_metrics_res
val_metrics_pd = performances_to_pd(val_metrics_dict)

Idx0 - model_layer1_multihead1_fold0.pkl: # layer = 1 | # heads = 1 | Fold = 0


  1%|▏         | 2/141 [00:00<00:08, 16.02it/s]

VAL Label info: Counter({0: 71837, 1: 71837})


100%|██████████| 141/141 [00:07<00:00, 17.94it/s]


tn = 66457, fp = 5380, fn = 5253, tp = 66584
y_pred: 0 = 71710 | 1 = 71964
y_true: 0 = 71837 | 1 = 71837
auc=0.9763|sensitivity=0.9269|specificity=0.9251|acc=0.9260|mcc=0.8520
precision=0.9252|recall=0.9269|f1=0.9261|aupr=0.9763
Idx1 - model_layer1_multihead1_fold1.pkl: # layer = 1 | # heads = 1 | Fold = 1


...

tn = 66542, fp = 5295, fn = 4306, tp = 67531
y_pred: 0 = 70848 | 1 = 72826
y_true: 0 = 71837 | 1 = 71837
auc=0.9815|sensitivity=0.9401|specificity=0.9263|acc=0.9332|mcc=0.8664
precision=0.9273|recall=0.9401|f1=0.9336|aupr=0.9799
Idx99 - model_layer2_multihead10_fold4.pkl: # layer = 2 | # heads = 10 | Fold = 4


  0%|          | 0/141 [00:00<?, ?it/s]

VAL Label info: Counter({0: 71837, 1: 71837})


100%|██████████| 141/141 [00:37<00:00,  3.81it/s]


tn = 66275, fp = 5562, fn = 3757, tp = 68080
y_pred: 0 = 70032 | 1 = 73642
y_true: 0 = 71837 | 1 = 71837
auc=0.9823|sensitivity=0.9477|specificity=0.9226|acc=0.9351|mcc=0.8706
precision=0.9245|recall=0.9477|f1=0.9359|aupr=0.9805


In [33]:
val_metrics_pd

,roc_auc,accuracy,mcc,f1,sensitivity,specificity,precision,recall,aupr
1-1-0,0.976315,0.925992,0.851986,0.926058,0.926876,0.925108,0.925240,0.926876,0.976251
1-1-1,0.978348,0.929159,0.858397,0.929635,0.935924,0.922394,0.923430,0.935924,0.977830
1-1-2,0.978418,0.927906,0.855912,0.928451,0.935521,0.920292,0.921487,0.935521,0.977886
1-1-3,0.978063,0.927266,0.854543,0.927450,0.929799,0.924732,0.925112,0.929799,0.977482
1-1-4,0.978182,0.928721,0.857450,0.928881,0.930983,0.926459,0.926790,0.930983,0.977465
1-2-0,0.976152,0.924517,0.849284,0.925423,0.936676,0.912357,0.914438,0.936676,0.976109
1-2-1,0.977653,0.926660,0.853403,0.927165,0.933586,0.919735,0.920832,0.933586,0.976971
1-2-2,0.978822,0.927753,0.855684,0.928482,0.937943,0.917563,0.919210,0.937943,0.977950
1-2-3,0.978650,0.928303,0.856716,0.928873,0.936314,0.920292,0.921549,0.936314,0.977943
1-2-4,0.979042,0.929813,0.859641,0.930017,0.932723,0.926904,0.927327,0.932723,0.977865


In [28]:
type_ = 'train'
save_ = False
use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

train_metrics_dict, train_ys_dict, train_attns_dict = {}, {}, {}
for idx, filename in enumerate(model_files):
    n_layers, n_heads, fold = model_layers_heads[idx]
    n_layers, n_heads, fold = int(n_layers), int(n_heads), int(fold)
    temp =  str(n_layers) + '-' + str(n_heads) + '-' + str(fold)
    print('Idx{} - {}: # layer = {} | # heads = {} | Fold = {}'.format(idx, filename, n_layers, n_heads, fold))
    
    model_eval = Transformer().to(device)
    criterion = nn.CrossEntropyLoss()
    model_pwd = '/home/chujunyi/5_ZY_MHC/model/pHLAIformer/'
    model_eval.load_state_dict(torch.load(model_pwd + filename), strict = True)

    if type_ == 'train' or type_ == 'val':
        data, pep_inputs, hla_inputs, labels, loader = data_with_loader(type_, fold,  batch_size)
    else:
        data, pep_inputs, hla_inputs, labels, loader = data_with_loader(type_, None, batch_size)
    print('{} Label info: {}'.format(str.upper(type_), Counter(data.label)))

    model_eval.eval()
    if save_:
        train_metrics_res, train_ys_res, train_attn_res = eval_step(model_eval, loader, labels, fold, type_, use_cuda, save_)
        train_metrics_dict[temp], train_ys_dict[temp], train_attns_dict[temp] = train_metrics_res, train_ys_res, train_attn_res
    else:
        train_metrics_res, _, _ = eval_step(model_eval, loader, labels, fold, type_, use_cuda, save_)
        train_metrics_dict[temp] = train_metrics_res
train_metrics_pd = performances_to_pd(train_metrics_dict)

Idx0 - model_layer1_multihead1_fold0.pkl: # layer = 1 | # heads = 1 | Fold = 0


  0%|          | 2/562 [00:00<00:36, 15.51it/s]

TRAIN Label info: Counter({0: 287329, 1: 287329})


100%|██████████| 562/562 [00:30<00:00, 18.42it/s]


tn = 267869, fp = 19460, fn = 18472, tp = 268857
y_pred: 0 = 286341 | 1 = 288317
y_true: 0 = 287329 | 1 = 287329
auc=0.9809|sensitivity=0.9357|specificity=0.9323|acc=0.9340|mcc=0.8680
precision=0.9325|recall=0.9357|f1=0.9341|aupr=0.9808
Idx1 - model_layer1_multihead1_fold1.pkl: # layer = 1 | # heads = 1 | Fold = 1


...

tn = 270455, fp = 16874, fn = 17161, tp = 270168
y_pred: 0 = 287616 | 1 = 287042
y_true: 0 = 287329 | 1 = 287329
auc=0.9860|sensitivity=0.9403|specificity=0.9413|acc=0.9408|mcc=0.8815
precision=0.9412|recall=0.9403|f1=0.9407|aupr=0.9860
Idx99 - model_layer2_multihead10_fold4.pkl: # layer = 2 | # heads = 10 | Fold = 4


  0%|          | 0/562 [00:00<?, ?it/s]

TRAIN Label info: Counter({1: 287348, 0: 287348})


100%|██████████| 562/562 [02:24<00:00,  3.90it/s]


tn = 270069, fp = 17279, fn = 14986, tp = 272362
y_pred: 0 = 285055 | 1 = 289641
y_true: 0 = 287348 | 1 = 287348
auc=0.9872|sensitivity=0.9478|specificity=0.9399|acc=0.9439|mcc=0.8877
precision=0.9403|recall=0.9478|f1=0.9441|aupr=0.9870


In [34]:
train_metrics_pd

,roc_auc,accuracy,mcc,f1,sensitivity,specificity,precision,recall,aupr
1-1-0,0.980921,0.933992,0.867989,0.934105,0.935711,0.932273,0.932505,0.935711,0.980809
1-1-1,0.980461,0.932024,0.864072,0.932276,0.935753,0.928295,0.928826,0.935753,0.980320
1-1-2,0.980316,0.930827,0.861706,0.931207,0.936355,0.925298,0.926115,0.936355,0.980211
1-1-3,0.980457,0.931749,0.863502,0.931648,0.930279,0.933219,0.933022,0.930279,0.980365
1-1-4,0.980689,0.932717,0.865439,0.932615,0.931202,0.934233,0.934033,0.931202,0.980482
1-2-0,0.981547,0.933124,0.866473,0.933879,0.944544,0.921703,0.923451,0.944544,0.981321
1-2-1,0.980263,0.930750,0.861520,0.930984,0.934142,0.927359,0.927848,0.934142,0.980192
1-2-2,0.981213,0.931839,0.863743,0.932252,0.937932,0.925747,0.926641,0.937932,0.981017
1-2-3,0.981469,0.932551,0.865137,0.932850,0.936999,0.928103,0.928737,0.936999,0.981324
1-2-4,0.982155,0.934687,0.869380,0.934567,0.932855,0.936519,0.936286,0.932855,0.981818


In [37]:
data_layer_head_metrics = dict()
data_layer_head_metrics['train'] = train_metrics_pd
data_layer_head_metrics['val'] = val_metrics_pd
data_layer_head_metrics['independent'] = independent_metrics_pd
data_layer_head_metrics['external'] = external_metrics_pd

np.save('./results/pHLAIformer_data_layer_head_metrics_pd_dict.npy', data_layer_head_metrics)

# 虽然layer=2效果更好，但是为了可解释性，还是选择1-9-4

In [31]:
types = ['train', 'val', 'independent', 'external']

model_file = 'model_layer1_multihead9_fold4.pkl'
model_layer_head = ['1', '9', '4']
n_layers, n_heads, fold = 1, 9, 4
temp =  str(n_layers) + '-' + str(n_heads) + '-' + str(fold)
print('{}: # layer = {} | # heads = {} | Fold = {}'.format(model_file, n_layers, n_heads, fold))

save_ = True
use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

metrics_dict, ys_dict, attns_dict = {}, {}, {}
for type_ in types:
    
    model_eval = Transformer().to(device)
    criterion = nn.CrossEntropyLoss()
    model_pwd = '/home/chujunyi/5_ZY_MHC/model/pHLAIformer/'
    model_eval.load_state_dict(torch.load(model_pwd + model_file), strict = True)

    if type_ == 'train' or type_ == 'val':
        data, pep_inputs, hla_inputs, labels, loader = data_with_loader(type_, fold,  batch_size)
    else:
        data, pep_inputs, hla_inputs, labels, loader = data_with_loader(type_, None, batch_size)
    print('{} Label info: {}'.format(str.upper(type_), Counter(data.label)))

    model_eval.eval()

    metrics_res, ys_res, attn_res = eval_step(model_eval, loader, labels, fold, use_cuda, save_)
    torch.save(attn_res, './results/pHLAIformer_{}_data_attn.pt'.format(type_))
    
    data['y_true'], data['y_pred'], data['y_prob'] = ys_res[0], ys_res[1], ys_res[2]
    metrics_dict[type_], ys_dict[type_] = metrics_res, data
    # attns_dict[type_] = attn_res

metrics_pd = performances_to_pd(metrics_dict)
np.save('./results/pHLAIformer_data_metrics_pd_dict.npy', metrics_dict)
np.save('./results/pHLAIformer_data_res_pd_dict.npy', ys_dict)
metrics_pd

model_layer1_multihead9_fold4.pkl: # layer = 1 | # heads = 9 | Fold = 4


  0%|          | 1/562 [00:00<01:31,  6.13it/s]

TRAIN Label info: Counter({1: 287348, 0: 287348})


100%|██████████| 562/562 [02:32<00:00,  3.68it/s]


tn = 270582, fp = 16766, fn = 18073, tp = 269275
y_pred: 0 = 288655 | 1 = 286041
y_true: 0 = 287348 | 1 = 287348
auc=0.9848|sensitivity=0.9371|specificity=0.9417|acc=0.9394|mcc=0.8788
precision=0.9414|recall=0.9371|f1=0.9392|aupr=0.9846


...

  0%|          | 0/102 [00:00<?, ?it/s]

EXTERNAL Label info: Counter({1: 51984, 0: 51881})


100%|██████████| 102/102 [00:14<00:00,  7.14it/s]


tn = 49362, fp = 2519, fn = 10108, tp = 41876
y_pred: 0 = 59470 | 1 = 44395
y_true: 0 = 51881 | 1 = 51984
auc=0.9505|sensitivity=0.8056|specificity=0.9514|acc=0.8784|mcc=0.7651
precision=0.9433|recall=0.8056|f1=0.8690|aupr=0.9552


,roc_auc,accuracy,mcc,f1,sensitivity,specificity,precision,recall,aupr
train,0.984848,0.939378,0.878766,0.939240,0.937104,0.941653,0.941386,0.937104,0.984597
val,0.980968,0.933078,0.866190,0.933374,0.937525,0.928630,0.929259,0.937525,0.979490
independent,0.978880,0.930844,0.861688,0.930927,0.930341,0.931348,0.931513,0.930341,0.978183
external,0.950488,0.878429,0.765103,0.868986,0.805556,0.951447,0.943259,0.805556,0.955213


In [32]:
len(attn_res), len(attn_res[0]), attn_res[0].shape

(103865, 9, torch.Size([9, 34, 15]))

# 把1-9-4数据都合并在一起来看attention

In [43]:
def eval_step_corrected(model, val_loader, labels, fold, use_cuda = False, save_ = False):
    device = torch.device("cuda" if use_cuda else "cpu")
    
    model.eval()
    torch.manual_seed(19961231)
    torch.cuda.manual_seed(19961231)
    with torch.no_grad():
        y_true_val_list, y_prob_val_list, dec_attns_val_list = [], [], []
        for val_pep_inputs, val_hla_inputs, val_labels in tqdm(val_loader):
            val_pep_inputs, val_hla_inputs, val_labels = val_pep_inputs.to(device), val_hla_inputs.to(device), val_labels.to(device)
            val_outputs, _, _, val_dec_self_attns = model(val_pep_inputs, val_hla_inputs)
            val_loss = criterion(val_outputs, val_labels)

            y_true_val = val_labels.cpu().numpy()
            y_prob_val = nn.Softmax(dim = 1)(val_outputs)[:, 1].cpu().detach().numpy()

            y_true_val_list.extend(y_true_val)
            y_prob_val_list.extend(y_prob_val)
            
            if save_:
                dec_attns_val_list.extend(val_dec_self_attns[0][:, :, 15:, :15]) # 只要（34,15）行HLA，列peptide
                
        assert (labels.numpy() == y_true_val_list).all()    
        y_pred_val_list = transfer(y_prob_val_list, threshold)
        ys_val = (y_true_val_list, y_pred_val_list, y_prob_val_list)
        metrics_val = performances(y_true_val_list, y_pred_val_list, y_prob_val_list, print_ = True)
        
        if save_: 
            return metrics_val, ys_val, dec_attns_val_list
        else:
            return metrics_val, ys_val, None

In [42]:
fold = 4
train_data = pd.read_csv('/home/chujunyi/5_ZY_MHC/Anthem/Dataset/train_data_fold{}.csv'.format(fold), index_col = 0)
val_data = pd.read_csv('/home/chujunyi/5_ZY_MHC/Anthem/Dataset/val_data_fold{}.csv'.format(fold), index_col = 0)
independent_data = pd.read_csv('/home/chujunyi/5_ZY_MHC/Anthem/Dataset/independent_set.csv', index_col = 0)
external_data = pd.read_csv('/home/chujunyi/5_ZY_MHC/Anthem/Dataset/external_set.csv', index_col = 0)

all_data = pd.concat([train_data, val_data, independent_data, external_data], axis = 0).drop_duplicates().reset_index(drop = True)

all_pep_inputs, all_hla_inputs, all_labels = make_data(all_data)
all_loader = Data.DataLoader(MyDataSet(all_pep_inputs, all_hla_inputs, all_labels), batch_size, shuffle = False, num_workers = 0)
print('Label info: {}'.format(Counter(all_data.label)))
all_data

Label info: Counter({0: 496538, 1: 425208})


,peptide,length,HLA,label,HLA_sequence
0,QRYLDELMKL,10,HLA-B*27:08,1,YHTEYREICAKTDESNLYLNYHDYTWAVLAYEWY
1,TEIPVAKI,8,HLA-B*49:01,1,YHTKYREISTNTYENIAYWRYNLYTWAELAYLWY
2,DNTPSEMPP,9,HLA-A*11:01,0,YYAMYQENVAQTDVDTLYIIYRDYTWAAQAYRWY
3,ARLPGVSRSGFS,12,HLA-B*27:05,1,YHTEYREICAKTDEDTLYLNYHDYTWAVLAYEWY
4,HLVDFQVTI,9,HLA-A*02:02,1,YFAMYGEKVAHTHVDTLYLRYHYYTWAVWAYTWY
...,...,...,...,...,...
921741,DAPTPGSNSTPGLR,14,HLA-B*18:01,0,YHSTYRNISTNTYESNLYLRYDSYTWAVLAYTWH
921742,PHLMPSNCGLEEKI,14,HLA-B*18:01,0,YHSTYRNISTNTYESNLYLRYDSYTWAVLAYTWH
921743,EYYRGLVPILFRNG,14,HLA-B*18:01,0,YHSTYRNISTNTYESNLYLRYDSYTWAVLAYTWH
921744,RSMLEVNYPMENGI,14,HLA-B*18:01,0,YHSTYRNISTNTYESNLYLRYDSYTWAVLAYTWH


In [44]:
type_ = 'all'

model_file = 'model_layer1_multihead9_fold4.pkl'
model_layer_head = ['1', '9', '4']
n_layers, n_heads, fold = 1, 9, 4
temp =  str(n_layers) + '-' + str(n_heads) + '-' + str(fold)
print('{}: # layer = {} | # heads = {} | Fold = {}'.format(model_file, n_layers, n_heads, fold))

save_ = False
use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

attns_dict = {}

model_eval = Transformer().to(device)
criterion = nn.CrossEntropyLoss()
model_pwd = '/home/chujunyi/5_ZY_MHC/model/pHLAIformer/'
model_eval.load_state_dict(torch.load(model_pwd + model_file), strict = True)

model_eval.eval()
metrics_res, ys_res, _ = eval_step_corrected(model_eval, all_loader, all_labels, fold, use_cuda, save_)
# torch.save(attn_res, './results/pHLAIformer_{}_data_attn.pt'.format(type_))

all_data['y_true'], all_data['y_pred'], all_data['y_prob'] = ys_res[0], ys_res[1], ys_res[2]
performances_to_pd(metrics_res)

  0%|          | 0/901 [00:00<?, ?it/s]

model_layer1_multihead9_fold4.pkl: # layer = 1 | # heads = 9 | Fold = 4


100%|██████████| 901/901 [02:02<00:00,  7.38it/s]


tn = 466254, fp = 30284, fn = 34163, tp = 391045
y_pred: 0 = 500417 | 1 = 421329
y_true: 0 = 496538 | 1 = 425208
auc=0.9783|sensitivity=0.9197|specificity=0.9390|acc=0.9301|mcc=0.8593
precision=0.9281|recall=0.9197|f1=0.9239|aupr=0.9757


,roc_auc,accuracy,mcc,f1,sensitivity,specificity,precision,recall,aupr
0,0.978267,0.930082,0.859259,0.92387,0.919656,0.93901,0.928123,0.919656,0.975684


In [45]:
all_corrected_data = all_data[all_data.y_pred == all_data.label].reset_index(drop = True)

all_corrected_pep_inputs, all_corrected_hla_inputs, all_corrected_labels = make_data(all_corrected_data)
all_corrected_loader = Data.DataLoader(MyDataSet(all_corrected_pep_inputs, all_corrected_hla_inputs, all_corrected_labels), batch_size, shuffle = False, num_workers = 0)
print('Label info: {}'.format(Counter(all_corrected_data.label)))

all_data_corrected.shape[0], all_data.shape[0]

Label info: Counter({0: 466254, 1: 391045})


(857299, 921746)

In [46]:
type_ = 'all_corrected'

model_file = 'model_layer1_multihead9_fold4.pkl'
model_layer_head = ['1', '9', '4']
n_layers, n_heads, fold = 1, 9, 4
temp =  str(n_layers) + '-' + str(n_heads) + '-' + str(fold)
print('{}: # layer = {} | # heads = {} | Fold = {}'.format(model_file, n_layers, n_heads, fold))

save_ = True
use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

attns_dict = {}

model_eval = Transformer().to(device)
criterion = nn.CrossEntropyLoss()
model_pwd = '/home/chujunyi/5_ZY_MHC/model/pHLAIformer/'
model_eval.load_state_dict(torch.load(model_pwd + model_file), strict = True)

model_eval.eval()
_, _, attn_res = eval_step_corrected(model_eval, all_corrected_loader, all_corrected_labels, fold, use_cuda, save_)
torch.save(attn_res, './results/pHLAIformer_{}_data_attn.pt'.format(type_))

  0%|          | 0/838 [00:00<?, ?it/s]

model_layer1_multihead9_fold4.pkl: # layer = 1 | # heads = 9 | Fold = 4


100%|██████████| 838/838 [02:15<00:00,  6.18it/s]


tn = 466254, fp = 0, fn = 0, tp = 391045
y_pred: 0 = 466254 | 1 = 391045
y_true: 0 = 466254 | 1 = 391045
auc=1.0000|sensitivity=1.0000|specificity=1.0000|acc=1.0000|mcc=1.0000
precision=1.0000|recall=1.0000|f1=1.0000|aupr=1.0000


In [47]:
all_corrected_data.to_csv('./results/pHLAIformer_all_corrected_data_res_pd.csv')

##### 文件太大了，改成float16

In [56]:
for i in trange(len(attn_res)):
    attn_res[i] = attn_res[i].half()

100%|██████████| 857299/857299 [00:26<00:00, 32135.01it/s]


In [57]:
attn_res[0].dtype

torch.float16

In [58]:
torch.save(attn_res, './results/pHLAIformer_all_corrected_data_attn_float16.pt')

# 模型

In [59]:
model_eval

Transformer(
  (pep_encoder): Encoder(
    (src_emb): Embedding(21, 64)
    (pos_emb): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): ModuleList(
      (0): EncoderLayer(
        (enc_self_attn): MultiHeadAttention(
          (W_Q): Linear(in_features=64, out_features=576, bias=False)
          (W_K): Linear(in_features=64, out_features=576, bias=False)
          (W_V): Linear(in_features=64, out_features=576, bias=False)
          (fc): Linear(in_features=576, out_features=64, bias=False)
        )
        (pos_ffn): PoswiseFeedForwardNet(
          (fc): Sequential(
            (0): Linear(in_features=64, out_features=512, bias=False)
            (1): ReLU()
            (2): Linear(in_features=512, out_features=64, bias=False)
          )
        )
      )
    )
  )
  (hla_encoder): Encoder(
    (src_emb): Embedding(21, 64)
    (pos_emb): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): ModuleList(
      (0

# 性能评估指标处理

### length, hla Results

In [144]:
def eval_performances_to_pd(metrics_length_dict, length):
    metrics_name = ['roc_auc', 'accuracy', 'mcc', 'f1', 'sensitivity', 'specificity', 'precision', 'recall', 'aupr', 'num']
    
    performances_pd = pd.DataFrame(metrics_length_dict[length], index = metrics_name).T
    performances_pd.loc['mean'] = performances_pd.mean(axis = 0)
    performances_pd.loc['median'] = performances_pd.median(axis = 0)
    performances_pd.loc['std'] = performances_pd.std(axis = 0)
    performances_pd.loc['sum', 'num'] = performances_pd['num'].sum(axis = 0)
    performances_pd['num'] = performances_pd['num'].astype('int')
    
    return performances_pd

In [145]:
def metrics_all_length_hla_pd_dict(all_data_res_pd_dict, data_type, lengths):
    data_eval = all_data_res_pd_dict[data_type]
    
    metrics_length_hla_dict, metrics_length_hla_pd_dict = {}, {}
    
    metrics_allsamples = performances(data_eval.y_true, data_eval.y_pred, data_eval.y_prob)
    metrics_length_hla_pd_dict['all'] = performances_to_pd(metrics_allsamples)

    for length in lengths:
        data_length = data_eval[data_eval.length == length]
        hlas_length = list(set(data_length.HLA))
        metrics_length_hla_dict[length] = {}
        for hla in tqdm(hlas_length):
            data_length_hla = data_length[data_length.HLA == hla]
            data_num = data_length_hla.shape[0]
            metrics_length_hla = performances(data_length_hla.y_true, data_length_hla.y_pred, data_length_hla.y_prob, print_ = False)
            metrics_length_hla_dict[length][hla] = list(metrics_length_hla) + [data_num]
        
    for length in lengths:
        metrics_length_hla_pd_dict[length] = eval_performances_to_pd(metrics_length_hla_dict, length)
    return metrics_length_hla_pd_dict

In [149]:
all_data_res_pd_dict = np.load('./results/pHLAIformer_data_res_pd_dict.npy', allow_pickle = True).item()

train_metrics_length_hla_dict = metrics_all_length_hla_pd_dict(all_data_res_pd_dict, 'train', list(range(8, 15)))
val_metrics_length_hla_dict = metrics_all_length_hla_pd_dict(all_data_res_pd_dict, 'val', list(range(8, 15)))
independent_metrics_length_hla_dict = metrics_all_length_hla_pd_dict(all_data_res_pd_dict, 'independent', list(range(8, 15)))
external_metrics_length_hla_dict = metrics_all_length_hla_pd_dict(all_data_res_pd_dict, 'external', list(range(8, 15)))

  0%|          | 0/48 [00:00<?, ?it/s]

tn = 270582, fp = 16766, fn = 18073, tp = 269275
y_pred: 0 = 288655 | 1 = 286041
y_true: 0 = 287348 | 1 = 287348
auc=0.9848|sensitivity=0.9371|specificity=0.9417|acc=0.9394|mcc=0.8788
precision=0.9414|recall=0.9371|f1=0.9392|aupr=0.9846


  0%|          | 0/111 [00:00<?, ?it/s]

tn = 66710, fp = 5127, fn = 4488, tp = 67349
y_pred: 0 = 71198 | 1 = 72476
y_true: 0 = 71837 | 1 = 71837
auc=0.9810|sensitivity=0.9375|specificity=0.9286|acc=0.9331|mcc=0.8662
precision=0.9293|recall=0.9375|f1=0.9334|aupr=0.9795


  0%|          | 0/111 [00:00<?, ?it/s]

tn = 79688, fp = 5874, fn = 5982, tp = 79894
y_pred: 0 = 85670 | 1 = 85768
y_true: 0 = 85562 | 1 = 85876
auc=0.9789|sensitivity=0.9303|specificity=0.9313|acc=0.9308|mcc=0.8617
precision=0.9315|recall=0.9303|f1=0.9309|aupr=0.9782


  0%|          | 0/5 [00:00<?, ?it/s]

tn = 49362, fp = 2519, fn = 10108, tp = 41876
y_pred: 0 = 59470 | 1 = 44395
y_true: 0 = 51881 | 1 = 51984
auc=0.9505|sensitivity=0.8056|specificity=0.9514|acc=0.8784|mcc=0.7651
precision=0.9433|recall=0.8056|f1=0.8690|aupr=0.9552


100%|██████████| 5/5 [00:00<00:00, 312.41it/s]


In [148]:
train_metrics_length_hla_dict[10]

,roc_auc,accuracy,mcc,f1,sensitivity,specificity,precision,recall,aupr,num
HLA-B*46:01,0.985868,0.945833,0.891791,0.945378,0.937500,0.954167,0.953390,0.937500,0.986822,480
HLA-A*02:07,0.987357,0.936644,0.873416,0.936097,0.928082,0.945205,0.944251,0.928082,0.988597,584
HLA-A*32:01,0.978150,0.917120,0.834242,0.917232,0.918478,0.915761,0.915989,0.918478,0.980591,736
HLA-A*26:01,0.988455,0.937500,0.876989,0.935323,0.903846,0.971154,0.969072,0.903846,0.984960,416
HLA-A*33:01,0.987386,0.935096,0.870686,0.936170,0.951923,0.918269,0.920930,0.951923,0.987276,832
HLA-B*18:01,0.980320,0.948529,0.898615,0.946970,0.919118,0.977941,0.976562,0.919118,0.985270,272
HLA-A*31:01,0.978880,0.929813,0.860463,0.931328,0.951872,0.907754,0.911652,0.951872,0.975219,1496
HLA-B*50:01,0.992836,0.963235,0.926872,0.962687,0.948529,0.977941,0.977273,0.948529,0.991865,272
HLA-C*07:02,0.977640,0.936047,0.872152,0.935673,0.930233,0.941860,0.941176,0.930233,0.973576,344
HLA-C*16:01,0.976374,0.908784,0.817736,0.907850,0.898649,0.918919,0.917241,0.898649,0.975128,296


##### 保存

In [150]:
metrics_data_length_hla_pd_dict = {}
metrics_data_length_hla_pd_dict['train'] = train_metrics_length_hla_dict
metrics_data_length_hla_pd_dict['val'] = val_metrics_length_hla_dict
metrics_data_length_hla_pd_dict['independent'] = independent_metrics_length_hla_dict
metrics_data_length_hla_pd_dict['external'] = external_metrics_length_hla_dict
np.save('./results/pHLAIformer_metrics_length_hla_pd_dict.npy', metrics_data_length_hla_pd_dict)

In [151]:
np.load('./results/pHLAIformer_metrics_length_hla_pd_dict.npy', allow_pickle = True).item()['external']['all']

,roc_auc,accuracy,mcc,f1,sensitivity,specificity,precision,recall,aupr
0,0.950488,0.878429,0.765103,0.868986,0.805556,0.951447,0.943259,0.805556,0.955213
